In [204]:
from joblib import load
import numpy as np
import pandas as pd
import spacy
import re
from fractions import Fraction
from decimal import Decimal, InvalidOperation
import unicodedata
import sys
sys.path.append("..")
from src.data import data_cleaning_util
from src.features import create_features
import matplotlib.pyplot as plt

### Plots of data

In [223]:
epi_data_df = pd.read_hdf('../data/processed/epi_vector.h5')


In [224]:
epi_data_df.head()

,""" 00 "" (soft wheat) flour * or cake flour",""" 00 "" flour",""" 00 "" flour, divide",""" > chicken stock chicken broth",""" > panko (japanese bread crumb bread crumb",""" baby "" Pattypan squash",""" biscuit cutter",""" brown candy "" sugar brown sugar",""" cauliflower floret (cut from 1 large head)",""" circular wire rack",...,ﬁne dry bread crumb,ﬁne salt,avg_rating,best_rating,worst_rating,prepare_again_rating,num_reviews,total_time,tags,title
http://www.epicurious.com/recipes/food/views/-adult-pimiento-cheese-100852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.5,4.0,0.0,0.85,25.0,0.0,"[Cheese, Vegetable, No-Cook, Vegetarian, Quick...","""Adult"" Pimiento Cheese"
http://www.epicurious.com/recipes/food/views/-an-apple-a-day-51133430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.5,4.0,0.0,0.67,3.0,0.0,"[Ginger, Dessert, Bake, Apple, Almond, Fall, R...","""An Apple a Day"""
http://www.epicurious.com/recipes/food/views/-blanketed-eggplant-305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,4.0,0.0,0.75,4.0,0.0,"[Tomato, Vegetable, Appetizer, Side, Vegetaria...","""Blanketed"" Eggplant"
http://www.epicurious.com/recipes/food/views/-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,4.0,0.0,1.00,2.0,0.0,"[Condiment/Spread, Tomato, Appetizer, Kid-Frie...","""Bloody Mary"" Tomato Toast with Celery and Hor..."
http://www.epicurious.com/recipes/food/views/-bow-tie-pasta-with-zucchini-101932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.5,4.0,0.0,0.93,48.0,0.0,"[Pasta, Vegetarian, Quick & Easy, Dinner, Basi...","""Bow-Tie"" Pasta with Zucchini"


In [228]:
# epi_data_df.describe()

### Random data

Following this linke:

https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

In [206]:
import random
df = pd.DataFrame()
wc = ['Bad','Good']
dc = ['Sober','Alcohol-impaired']
tv = ['None','Disobey stop sign','Exceed speed limit','Disobey traffic signal']
sb = ['yes','no']
cs = ['Minor','Major']
df['Weather Condition'] = np.random.choice(wc,1000,p=[0.4,0.6]) 
df['Driver Condition'] = np.random.choice(dc,1000,p=[0.5,0.5]) 
df['Traffic Violation'] = np.random.choice(tv,1000,p=[0.1,0.4,0.3,0.2])
df['Seat Belt'] = np.random.choice(sb,1000,p=[0.7,0.3])
df['Crash Severity'] = np.random.choice(cs,1000,p=[0.6,0.4])
# df.loc[df['Weather Condition']=='Good']

def weight_prob(row):
    if row["Driver Condition"] == 'Alcohol-impaired' and np.random.random() >=0.4:
        row["Crash Severity"] = 'Major'
    return row
df.apply(weight_prob, axis=1)
df

,Weather Condition,Driver Condition,Traffic Violation,Seat Belt,Crash Severity
0,Good,Sober,None,yes,Minor
1,Good,Alcohol-impaired,Disobey traffic signal,no,Minor
2,Bad,Alcohol-impaired,Disobey stop sign,yes,Major
3,Good,Sober,None,no,Minor
4,Bad,Alcohol-impaired,Disobey stop sign,yes,Major
...,...,...,...,...,...
995,Good,Alcohol-impaired,Exceed speed limit,no,Major
996,Good,Alcohol-impaired,Disobey stop sign,yes,Major
997,Good,Alcohol-impaired,Disobey stop sign,yes,Major
998,Good,Sober,Exceed speed limit,yes,Minor


In [207]:
# df['Crash Severity'].loc[df['Crash Severity']=='Major'] =1
# df['Crash Severity'].loc[df['Crash Severity']=='Minor'] =0
# df = pd.read_csv('iris.csv')
# df

In [208]:
# Seperate data from attribute we will used to classify, 
# which is class.

X = df.drop('Crash Severity', axis=1)
Y = df['Crash Severity']
print(Y.head())
X.head()

0    Minor
1    Minor
2    Major
3    Minor
4    Major
Name: Crash Severity, dtype: object


,Weather Condition,Driver Condition,Traffic Violation,Seat Belt
0,Good,Sober,None,yes
1,Good,Alcohol-impaired,Disobey traffic signal,no
2,Bad,Alcohol-impaired,Disobey stop sign,yes
3,Good,Sober,None,no
4,Bad,Alcohol-impaired,Disobey stop sign,yes


In [209]:
from sklearn import preprocessing
# label encoder
le = preprocessing.LabelEncoder()

# Converting string labels into numbers

Y = le.fit_transform(Y)
weather_encoded = le.fit_transform(X['Weather Condition'])
X['Weather_Condition'] = weather_encoded
driver_encoded = le.fit_transform(X['Driver Condition'])
X['Driver_Condition'] = driver_encoded
traffic_encoded = le.fit_transform(X['Traffic Violation'])
X['Traffic_Violation'] = traffic_encoded
seat_encoded = le.fit_transform(X['Seat Belt'])
X['Seat_Belt'] = seat_encoded
# print(X.Weather_Condition)
X

,Weather Condition,Driver Condition,Traffic Violation,Seat Belt,Weather_Condition,Driver_Condition,Traffic_Violation,Seat_Belt
0,Good,Sober,None,yes,1,1,3,1
1,Good,Alcohol-impaired,Disobey traffic signal,no,1,0,1,0
2,Bad,Alcohol-impaired,Disobey stop sign,yes,0,0,0,1
3,Good,Sober,None,no,1,1,3,0
4,Bad,Alcohol-impaired,Disobey stop sign,yes,0,0,0,1
...,...,...,...,...,...,...,...,...
995,Good,Alcohol-impaired,Exceed speed limit,no,1,0,2,0
996,Good,Alcohol-impaired,Disobey stop sign,yes,1,0,0,1
997,Good,Alcohol-impaired,Disobey stop sign,yes,1,0,0,1
998,Good,Sober,Exceed speed limit,yes,1,1,2,1


In [210]:
x = list(zip(X.Weather_Condition,X.Driver_Condition,X.Traffic_Violation))
# x

In [211]:
x

[(1, 1, 3),
 (1, 0, 1),
 (0, 0, 0),
 (1, 1, 3),
 (0, 0, 0),
 (0, 1, 0),
 (0, 0, 2),
 (1, 0, 0),
 (0, 0, 3),
 (1, 1, 2),
 (1, 1, 2),
 (0, 0, 2),
 (0, 1, 2),
 (1, 1, 0),
 (1, 1, 3),
 (1, 0, 2),
 (1, 0, 0),
 (1, 1, 0),
 (1, 0, 0),
 (0, 1, 3),
 (1, 1, 2),
 (0, 1, 2),
 (1, 0, 0),
 (1, 1, 1),
 (1, 1, 1),
 (0, 0, 1),
 (0, 0, 3),
 (1, 0, 1),
 (0, 0, 1),
 (1, 0, 0),
 (0, 0, 3),
 (1, 1, 0),
 (1, 1, 0),
 (1, 0, 0),
 (0, 0, 0),
 (0, 1, 2),
 (0, 0, 3),
 (1, 1, 0),
 (1, 1, 2),
 (1, 1, 2),
 (0, 1, 0),
 (0, 0, 0),
 (1, 1, 2),
 (1, 0, 0),
 (1, 1, 0),
 (0, 0, 2),
 (1, 1, 0),
 (1, 1, 0),
 (0, 0, 0),
 (0, 1, 3),
 (1, 1, 0),
 (1, 0, 1),
 (0, 1, 1),
 (1, 0, 2),
 (0, 1, 3),
 (1, 1, 2),
 (0, 0, 0),
 (0, 1, 0),
 (0, 1, 1),
 (1, 1, 1),
 (1, 1, 0),
 (1, 1, 1),
 (0, 1, 2),
 (1, 0, 2),
 (1, 1, 0),
 (1, 1, 0),
 (1, 0, 0),
 (0, 1, 0),
 (0, 0, 1),
 (0, 1, 0),
 (1, 1, 0),
 (1, 1, 2),
 (0, 1, 2),
 (1, 0, 0),
 (0, 0, 0),
 (1, 1, 1),
 (1, 0, 0),
 (1, 0, 3),
 (0, 0, 1),
 (1, 0, 1),
 (1, 0, 2),
 (1, 0, 2),
 (1, 1, 2),
 (1,

In [212]:
from sklearn.naive_bayes import GaussianNB

# create a guassian classifier
model = GaussianNB()
model.fit(x,Y)
print(model.epsilon_)

1.093279000000004e-09


In [213]:
# predict outputs
v = (5.1, 3.5, 1.4, 0.2) # 
predicted = model.predict(x)
print("Predicted value ", predicted)

Predicted value  [1 0 0 1 0 1 0 0 0 1 1 0 1 1 1 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 1 0
 1 1 1 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0
 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 1 0 0 1 0 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1
 0 1 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1
 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0
 0 1 1 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 1 0 1 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0
 1 1 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 1
 1 1 0 0 0 1 0 0 1 0 0 1 0 1 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0
 0 1 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 1 1 1 0 1 1 0 1
 0 0 1 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 0 1 0
 0 0 0 0 1 1 0 1 0 0 1 1 1 1 0 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1
 0 1 0 0

In [214]:
# from sklearn.model_selection import train_test_split
# X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [215]:
# from sklearn.svm import SVC as svc
# svclassifier = svc(kernel='linear')
# svclassifier.fit(X_train,Y_train)

In [216]:
# plt.scatter(df['Weather Condition'],df['Crash Severity'])
# df['Weather Condition'].corr(df['Crash Severity'])

In [217]:
# epi_data_df.describe()
# epi_data_df.shape()

In [218]:
# Can't read epi_features.pickle
# epi_features_df = pd.read_pickle('../data/interim/epi_features.pickle')


In [219]:
# epi_ingredients = pd.read_pickle('../data/interim/epi_ingredients.pickle')

In [220]:
# plt.scatter(epi_data_df[:1000]['title'],epi_data_df[:1000]['num_reviews'].sort_values(ascending=False))
# plt.show()